In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [6]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 110, 5)
non_lin = [1]

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 0:
            nonlin = tf.exp
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 2, reg = 'l2', 
                                lr = 1e-4, train_params = False, scale_init = 1, bias_init = 0)
        
        loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                             max_iters = 200) 
        
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                  100%
[#                     ] | ETA: 00:00:00

new best model for cell  517442753 . r2 test:  -0.0121992607865
new best model for cell  517444738 . r2 test:  0.0152964662532
new best model for cell  517443331 . r2 test:  0.0234830558252
new best model for cell  517442436 . r2 test:  0.0430124977009
new best model for cell  517445575 . r2 test:  0.0166415565976
new best model for cell  517444426 . r2 test:  0.0131889012692
new best model for cell  517442471 . r2 test:  -0.0104962223737
new best model for cell  517443181 . r2 test:  -0.011561032389
new best model for cell  517443087 . r2 test:  0.0165223704613
new best model for cell  517446833 . r2 test:  0.0593811177061
new best model for cell  517443827 . r2 test:  -0.0538711993743
new best model for cell  517443924 . r2 test:  -0.0500632470467
new best model for cell  517443957 . r2 test:  0.0481849530491
new best model for cell  517442722 . r2 test:  -0.025770218487
new best model for cell  517446488 . r2 test:  0.051223825618
new best model for cell  517442675 . r2 test:  0.043

[###                   ] | ETA: 00:31:07

 517443571 . r2 test:  0.00534590819422
new best model for cell  517442753 . r2 test:  0.0324677051336
new best model for cell  517445575 . r2 test:  0.0190367770004
new best model for cell  517444426 . r2 test:  0.017922223832
new best model for cell  517444175 . r2 test:  0.0265397667299
new best model for cell  517444738 . r2 test:  0.0432737871558
new best model for cell  517444426 . r2 test:  0.0315664938317
new best model for cell  517442471 . r2 test:  -0.00424594600715
new best model for cell  517443181 . r2 test:  0.012064146641
new best model for cell 

[#####                 ] | ETA: 00:58:51

...

[##################    ] | ETA: 00:44:45

 517444175 . r2 test:  0.0367556178721
new best model for cell  517444738 . r2 test:  0.0589949430966
new best model for cell  517445575 . r2 test:  0.0567054463131
new best model for cell  517442753 . r2 test:  0.072997532184
new best model for cell  517445575 . r2 test:  0.0651040130893
new best model for cell 

[######################] | ETA: 00:00:00

 517443827 . r2 test:  0.0721977517522
new best model for cell  517446833 . r2 test:  0.0739406503966
new best model for cell  517442675 . r2 test:  0.0599556327036



Total time elapsed: 05:09:57


In [15]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]

explained var:  0.205 non_lin: 0 n_components 1
(0.20471333787080059, (0, 1))
explained var:  0.067 non_lin: 0 n_components 1
(0.067399138643032402, (0, 1))
explained var:  -0.012 non_lin: 0 n_components 61
(-0.01225000620757246, (0, 61))
explained var:  0.321 non_lin: 0 n_components 1
(0.32066256818647759, (0, 1))
explained var:  0.243 non_lin: 0 n_components 21
(0.24282624633409033, (0, 21))
explained var:  0.174 non_lin: 0 n_components 1
(0.17381542853821436, (0, 1))
explained var:  0.047 non_lin: 0 n_components 1
(0.046700509571936236, (0, 1))
explained var:  0.012 non_lin: 0 n_components 81
(0.011955503314439762, (0, 81))
explained var:  0.112 non_lin: 0 n_components 1
(0.11185500602632083, (0, 1))
explained var:  0.263 non_lin: 0 n_components 21
(0.26327412441769821, (0, 21))
explained var:  0.005 non_lin: 0 n_components 61
(0.004968124413065044, (0, 61))
explained var:  0.182 non_lin: 0 n_components 1
(0.18216877669256215, (0, 1))
explained var:  0.118 non_lin: 0 n_components 1


In [ ]:
pkl_file = open('./boc/formatted/LNE_rs_sig', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()